### Import

In [53]:
import os
import numpy as np
import sys
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import spectral.io.aviris as aviris
import matplotlib.tri as mtri
import trimesh
import pyrender
import pickle
import numpy as np
import warnings
import pywavefront
import open3d as o3d

from stl import mesh
from tqdm import tqdm
from PIL import Image
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
from scipy.spatial.distance import squareform
from scipy.stats import multivariate_normal as mnorm
from scipy import ndimage
from matplotlib import cm
from mpl_toolkits import mplot3d
from skimage import io, color
from scipy.spatial.transform import Rotation as rot
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

### Fonts

In [54]:
warnings.filterwarnings('ignore')
csfont = {'fontname':'Georgia'}
hfont = {'fontname':'Helvetica'}

### Read All OBJ files generated by FaceGen
Read all the OBJ model files, but remove the meshes for the teeth and tongue to simplify our demo

In [63]:
models = []
sets = ['set1','set2','set3']
for curset in sets:
    for i in range(100):
        mtlfile = ''
        v = []
        vn = []
        vt = []
        fv = []
        fvn = []
        fvt = []
        materials = []
        object_ids = []
        group_names = []
        vertices = []
        normals = []
        uvcoords = []
        faces_vert = []
        faces_norm = []
        faces_text = []
        with open('../../data/facegen/obj/' + curset + '/' + str(i) + '.obj') as f:
            for line in f:

                #Read and split
                line = line.rstrip()
                elements = line.split(' ')

                #Store corresponding mtl file
                if (elements[0] == 'mtllib'): 
                    mtlfile = elements[1]

                #Get Mesh Id
                if (elements[0] == 's'): 
                    object_ids.append(elements[1])

                #Get Mesh Name
                if (elements[0] == 'g'):
                    vertices.append(np.array(v))
                    normals.append(vn)
                    uvcoords.append(vt)
                    faces_vert.append(np.array(fv)-1)
                    faces_norm.append(np.array(fvn)-1)
                    faces_text.append(np.array(fvt)-1)
                    group_names.append(elements[1])
                    v = []
                    vn = []
                    vt = []
                    fv = []
                    fvn = []
                    fvt = []

                #Unpack the vertices
                if (elements[0] == 'v'): 
                    v.append(np.array(elements[1:]).astype(float))

                #Unpack the normals
                if (elements[0] == 'vn'): 
                    vn.append(np.array(elements[1:]).astype(float))

                #Unpack the texture coords
                if (elements[0] == 'vt'): 
                    vt.append(np.array(elements[1:]).astype(float))

                #Unpack the faces
                if (elements[0] == 'f'): 
                    if (len(elements) == 5):
                        v1_elem = np.array(elements[1].split('/')).astype(int)
                        v2_elem = np.array(elements[2].split('/')).astype(int)
                        v3_elem = np.array(elements[3].split('/')).astype(int)
                        v4_elem = np.array(elements[4].split('/')).astype(int)
                        fv.append(np.array([v1_elem[0],v2_elem[0],v3_elem[0],v4_elem[0]]))
                        fvt.append(np.array([v1_elem[1],v2_elem[1],v3_elem[1],v4_elem[1]]))
                        fvn.append(np.array([v1_elem[2],v2_elem[2],v3_elem[2],v4_elem[2]]))

            #Remove unnecessary meshes and store
            faces_vert = faces_vert[1:4]
            faces_norm = faces_norm[1:4]
            faces_text = faces_text[1:4]
            group_names = group_names[1:4]
            vertices = vertices[1:4]
            normals = normals[1:4]
            uvcoords = uvcoords[1:4]
            models.append([vertices,normals,uvcoords,faces_vert,faces_norm,faces_text,group_names])

### Load Corresponding Texture Images as well

In [64]:
eyeL_textures = []
eyeR_textures = []
skin_textures = []
skin_textures_64 = []
skin_textures_96 = []
skin_textures_128 = []
sets = ['set1','set2','set3']
for curset in sets:
    for i in range(100):
        eyeL_loadname = '../../data/facegen/obj/' + curset + '/' + str(i) + '_eyeL_hi.jpg'
        eyeR_loadname = '../../data/facegen/obj/' + curset + '/' + str(i) + '_eyeL_hi.jpg'
        skin_loadname = '../../data/facegen/obj/' + curset + '/' + str(i) + '_skin_hi.jpg'
        eyeL_texture = Image.open(eyeL_loadname)
        eyeR_texture = Image.open(eyeR_loadname)
        skin_texture = Image.open(skin_loadname)

        skin_texture_64 = skin_texture.resize((64,64))
        skin_texture_96 = skin_texture.resize((96,96))
        skin_texture_128 = skin_texture.resize((128,128))
        
        eyeL_texture = np.array(eyeL_texture)
        eyeR_texture = np.array(eyeR_texture)
        skin_texture = np.array(skin_texture)
        skin_texture_64 = np.array(skin_texture_64)
        skin_texture_96 = np.array(skin_texture_96)
        skin_texture_128 = np.array(skin_texture_128)

        eyeL_textures.append(eyeL_texture)
        eyeR_textures.append(eyeR_texture)
        skin_textures.append(skin_texture)

        skin_textures_128.append(skin_texture_128)
        skin_textures_96.append(skin_texture_96)
        skin_textures_64.append(skin_texture_64)

eyeL_textures = np.array(eyeL_textures)
eyeR_textures = np.array(eyeR_textures)
skin_textures = np.array(skin_textures)
skin_textures_64 = np.array(skin_textures_64)
skin_textures_96 = np.array(skin_textures_96)
skin_textures_128 = np.array(skin_textures_128)

### Create Matrices for PCA and tSNE

In [65]:
XV = []
for model in models:
    [v,n,uv,fv,fn,ft,groupnames] = model
    V = np.zeros((1,3))
    for i in range(len(v)):
        vertices = v[i]
        V = np.vstack((V,vertices))
    column = np.reshape(V,(V.shape[0]*V.shape[1]))
    XV.append(column)
XV = np.array(XV)

### Pickle Data

In [66]:
X = {}
X['XV'] = XV
X['models'] = models
X['eyeL_textures'] = eyeL_textures
X['eyeR_textures'] = eyeR_textures
X['skin_textures_256'] = skin_textures
X['skin_textures_64'] = skin_textures_64
X['skin_textures_96'] = skin_textures_96
X['skin_textures_128'] = skin_textures_128
pickle.dump( X, open( "../../data/facegen/pickle/save.p","wb"))